In [ ]:
import tensorflow as tf 
import tensorflow_hub as hub
import tensorflow_text as text 

import pandas as pd 
df = pd.read_csv(r'/spam.csv')
df.head(10)

In [ ]:
df.groupby('Category').discribe()

In [ ]:
df['spam'] = df['Category'].apply(lambda x: 1 if x=='spam' else 0)
df.head(10)

In [ ]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(df['Message'], df['spam'], stratify=df['spam'])

In [ ]:
x_train.shape , x_test.shape 


In [ ]:
x_train.head(4)

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_sentence_embedding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pt']

get_sentence_embedding(
    [
        "500$ discount. hurry up", 
        "Bhavin, are you up for a volleybal game tomorrow?"
    ]
)

In [ ]:
embed = get_sentence_embedding(
    [
        "banana", 
        "grapes",
        "mango",
        "jeff bezos",
        "elon musk",
        "bill gates",
    ]
)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([embed[0]], [embed[1]])

In [ ]:
# Bert layers
text_input = tf.keras.Input(shape=() , dtype=tf.string , name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
# Neural network layers
l = tf.keras.layers.Dropout(0.1 , name='dropout')(outputs['pooled_output'])
l = tf.keras.layers.Dense(units=1 , activation='sigmoid' , name='output')(1)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

In [ ]:
len(x_train)

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=5)

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
reviews = [
    'Reply to win Â£100 weekly! Where will the 2006 FIFA World Cup be held? Send STOP to 87239 to end service',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]
model.predict(reviews)